In [ ]:
%pip install python-dotenv
%pip install openai==1.13.3
%pip install -qU langchain-openai
%pip install neo4j
%pip install langchain langchain-community

In [1]:
from dotenv import load_dotenv
import os
from openai import AzureOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain_openai import AzureChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import LLMChain

In [2]:
load_dotenv()

# Azure OpenAI variables
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT =os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

### Connecting to the graph and making a Cypher query

In [3]:
#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [6]:
#Quering the graph unsing Cypher
result = graph.query("""
MATCH (t:Thesis)-[:advisor]-(a:Person) MATCH (a {label: 'Silva, Jamile Borges da'}) RETURN t.title, t.abstract;
""")

In [7]:
result

[{'t.title': 'A implementação das leis 10.639/2003 e a 11.645/2008 na Rede de Ensino de Lapão: uma proposta de intervenção curricular intercultural',
  't.abstract': 'Este Projeto de Intervenção estuda a implementação das Leis 10.639/2003 e 11.645/2008 na Rede Municipal de Ensino de Lapão-Ba, na perspectiva do currículo intercultural e representa uma continuidade dos estudos e pesquisas que realizo na área de educação e diversidade. Teve como objetivo de pesquisa, a investigação de duas situações principais: primeiro, a análise de como o trabalho pedagógico, realizado no ambiente escolar, com relação à temática pode ser considerado processo de efetivação da legislação em estudo. Nessa abordagem a escola é vista como campo propício para afirmação de uma construção identitária reconhecendo que nossa identidade é também social e culturalmente construída. Segundo, investigar como o município de Lapão está organizado em termos legais e com relação à política de formação continuada para prof

In [8]:
# Printing the graph schema
graph.refresh_schema()
print(graph.schema)

Node properties:
Resource {uri: STRING, label: STRING, givenname: STRING, family_name: STRING, acronym: STRING, scopeNote: STRING, repository: STRING, created: STRING, abstract: STRING, title: STRING, identifier: STRING}
_GraphConfig {_dataTypePropertyLabel: STRING, _subPropertyOfRel: STRING, _classNamePropName: STRING, _handleVocabUris: INTEGER, _applyNeo4jNaming: BOOLEAN, _relNamePropName: STRING, _domainRel: STRING, _keepLangTag: BOOLEAN, _keepCustomDataTypes: BOOLEAN, _handleMultival: INTEGER, _objectPropertyLabel: STRING, _rangeRel: STRING, _classLabel: STRING, _handleRDFTypes: INTEGER, _subClassOfRel: STRING}
Program {uri: STRING, label: STRING}
NamedIndividual {family_name: STRING, label: STRING, givenname: STRING, uri: STRING, acronym: STRING, created: STRING, abstract: STRING, title: STRING, repository: STRING, identifier: STRING}
Department {label: STRING, uri: STRING}
DatatypeProperty {uri: STRING, scopeNote: STRING, isDefinedBy: STRING, term_status: STRING, label: STRING, c

### Using a cypher query to ground the LLM

In [9]:
# Connection to the Azure OpenAI model
client = AzureOpenAI(
  azure_endpoint=AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version=OPENAI_API_VERSION
)


chat_llm = AzureChatOpenAI(deployment_name=OPENAI_DEPLOYMENT_NAME,temperature=0)

In [10]:
# Using information queried from the knowledge graph
ground_information =  graph.query(""" MATCH (t:Thesis)-[:advisor]-(a:Person) MATCH (a {label: 'Silva, Jamile Borges da'}) RETURN t.title, t.abstract;""")
#ground_information =  graph.query(""" MATCH (n:Wellbore {WellboreName: "31/2-22 S"}) RETURN n;""")


prompt = PromptTemplate(template="""You are a bot assistant having a conversation about academic thesis and dissertations.

Chat History: {chat_history}
Context: {context}
Question: {question}
""", input_variables=["chat_history", "context", "question"])

In [11]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)

chat_chain = LLMChain(llm=chat_llm, prompt=prompt, memory=memory) #LLMChain

while True:
    question = input("> ")
    print(question)
    response = chat_chain.invoke({
        "context": ground_information,
        "question": question
        })
    print(response["text"])

C:\Users\facordei\AppData\Local\Temp\ipykernel_23892\767680771.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
C:\Users\facordei\AppData\Local\Temp\ipykernel_23892\767680771.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chat_chain = LLMChain(llm=chat_llm, prompt=prompt, memory=memory) #LLMChain


What is the titles of the thesis in your contex?
The titles of the thesis in my context are:

1. "A implementação das leis 10.639/2003 e a 11.645/2008 na Rede de Ensino de Lapão: uma proposta de intervenção curricular intercultural"
2. "“Entre a rua e o ciberespaço”: ciberracismo nas redes sociais brasileiras"
3. "Método Colhetear: a heteroidentificação como princípio educativo"
4. "Necroeducação: Racismo, juventude e enfrentamento na escola pública em Salvador"
5. "Uma intervenção formativa para as relações étnico-raciais no curso de Licenciatura em Pedagogia do IFES Campus Itapina"
Summarize the abstract in your context. Give the answer in Portuguease.
Os títulos das teses em meu contexto são sobre temas relacionados à educação e diversidade, ciberracismo nas redes sociais brasileiras, heteroidentificação como princípio educativo, práticas docentes sobre a política de enfrentamento ao racismo em escolas públicas de Salvador e uma intervenção formativa para as relações étnico-raciais 

In [29]:
question = input("> ")
#print (question)


In [6]:
# Connection to the Azure OpenAI model
client = AzureOpenAI(
  azure_endpoint=AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version=OPENAI_API_VERSION
)


llm = AzureChatOpenAI(deployment_name=OPENAI_DEPLOYMENT_NAME,temperature=0)

### Using LLM to query the graph using GraphCypherQAChain library

In [9]:
# FIRST PROMPT - Transform a natural language query in a Cypher query
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about the offshore oil and gas industry.
Convert the user's question based on the schema.
Do not include any text except the generated Cypher statement.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
The property existence syntax `... exists(variable.property)` is no longer supported. Please use `variable.property IS NOT NULL` instead.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

# Memory for chat conversation
memory = ConversationBufferMemory(memory_key="chat_history", input_key="query", return_messages=True)

# SECOND PROMPT - Use the Cypher query to answer the user query
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    memory=memory
)


# Chat loop
while True:
    question = input("> ")
    response = cypher_chain.invoke({
        "query": question
        })
    
    print(response['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
What are the names of the facilities that are responsible for drilling wellbore with NpdidWellbore '5693-L-008'? 

Cypher statement:
MATCH (:Wellbore {NpdidWellbore: '5693-L-008'})<-[:DRILLED]-(f:Facility) WHERE EXISTS((:Company)-[:IS_RESPONSIBLE_FOR]->(f)) RETURN f.Name


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'What': expected
  "ALTER"
  "CALL"
  "CREATE"
  "DEALLOCATE"
  "DELETE"
  "DENY"
  "DETACH"
  "DROP"
  "DRYRUN"
  "ENABLE"
  "FINISH"
  "FOREACH"
  "GRANT"
  "INSERT"
  "LOAD"
  "MATCH"
  "MERGE"
  "NODETACH"
  "OPTIONAL"
  "REALLOCATE"
  "REMOVE"
  "RENAME"
  "RETURN"
  "REVOKE"
  "SET"
  "SHOW"
  "START"
  "STOP"
  "TERMINATE"
  "UNWIND"
  "USE"
  "USING"
  "WITH" (line 1, column 1 (offset: 0))
"What are the names of the facilities that are responsible for drilling wellbore with NpdidWellbore '5693-L-008'?"
 ^}

-   How many facilities belongs to the "GULLFAKS" oil field?
-   Give me the name and FactPage URL of the facilities of the "GULLFAKS" oil field?